### Sentiment Analysis of Yelp Reviews using BERT 

In [1]:
#Install Pytorch
!pip3 install torch torchvision torchaudio

In [3]:
#Intsall dependencies
!pip install transformers requests beautifulsoup4 pandas numpy

In [42]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd

In [5]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')


/Users/tm/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [28]:
tokens= tokenizer.encode('I loved this place', return_tensors= 'pt')

In [29]:
tokens

tensor([[  101,   151, 46747, 10372, 11125,   102]])

In [30]:
results =  model(tokens)

In [31]:
int(torch.argmax(results.logits))+1

5

In [32]:
#Collect Reviews

In [38]:
r= requests.get('https://www.yelp.com/biz/mercurios-shadyside-pittsburgh')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class': regex})
reviews = [result.text for result in results]

In [40]:
results[0].text

"Started my Pittsburgh foodie adventures in this adorable little sit-down pizza restaurant in Shadyside! We tried the Pizza Del Re (white pizza) and Pizza Della Casa (red pizza), and both lived up to the hype! Pizza Del Re:- White pizza with mozzarella, truffle spread, & prosciutto (it comes with mushrooms too, but we got it without because my boyfriend doesn't like mushrooms) - A super rich truffle taste that was so yummy - Loved the thin crust, but there were parts that tasted a little too burntPizza Della Casa:- Red tomato sauce pizza with mozzarella, grape tomatoes, arugula, prosciutto, & parmigiano - I love the combination of prosciutto & arugula and it was a plus that the arugula tasted so fresh & wasn't bitter- I personally prefer red sauce over white sauce, so I liked this pizza a little bit more To top off your meal here, make sure to get their gelato for dessert! I got a small size with 2 flavors -- tiramisu and hazelnut. Both flavors were super rich and yummy and definitely 

In [41]:
reviews

["Started my Pittsburgh foodie adventures in this adorable little sit-down pizza restaurant in Shadyside! We tried the Pizza Del Re (white pizza) and Pizza Della Casa (red pizza), and both lived up to the hype! Pizza Del Re:- White pizza with mozzarella, truffle spread, & prosciutto (it comes with mushrooms too, but we got it without because my boyfriend doesn't like mushrooms) - A super rich truffle taste that was so yummy - Loved the thin crust, but there were parts that tasted a little too burntPizza Della Casa:- Red tomato sauce pizza with mozzarella, grape tomatoes, arugula, prosciutto, & parmigiano - I love the combination of prosciutto & arugula and it was a plus that the arugula tasted so fresh & wasn't bitter- I personally prefer red sauce over white sauce, so I liked this pizza a little bit more To top off your meal here, make sure to get their gelato for dessert! I got a small size with 2 flavors -- tiramisu and hazelnut. Both flavors were super rich and yummy and definitely

In [44]:
df = pd.DataFrame(np.array(reviews), columns= ['reviews'])

In [46]:
df.tail()

,reviews
5,Love Mercurio's! The pizza is delicious and th...
6,despite being someone who normally dislikes pi...
7,I was very excited to try this restaurant. Whe...
8,I recommend this place - BUT - dine in. These ...
9,Love their pizza! The crust is magical and del...


In [51]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors= 'pt')
    results = model(tokens)
    return int(torch.argmax(results.logits))+1

In [52]:
sentiment_score(df['reviews'].iloc[1])

4

In [53]:
df['sentiment_score'] = df['reviews'].apply(lambda x: sentiment_score(x[:512]))

In [54]:
df

,reviews,sentiment_score
0,Started my Pittsburgh foodie adventures in thi...,4
1,"Okay, to begin, the pizza is delicious! We had...",4
2,Really excellent Neapolitan pizza in a homey s...,5
3,We had caprese (my friend loves it so much but...,5
4,Mercurio's is in every way perfect. That's rea...,5
5,Love Mercurio's! The pizza is delicious and th...,5
6,despite being someone who normally dislikes pi...,5
7,I was very excited to try this restaurant. Whe...,1
8,I recommend this place - BUT - dine in. These ...,3
9,Love their pizza! The crust is magical and del...,5
